In [1]:
#import Pkg;Pkg.activate("convex")
import Pkg;Pkg.activate("nlp")
Pkg.instantiate()
# Pkg.resolve()
# Pkg.gc()
# Pkg.precompile()

  Activating environment at `~/SageMaker/networks_hub/SAN/code/network_simulations/nlp/Project.toml`
Precompiling project...
  ✓ OpenSpecFun_jll
  ✓ Bzip2_jll
  ✓ NLopt_jll
  ✓ fzf_jll
  ✓ CoinUtils_jll
  ✓ CodecBzip2
  ✓ JLFzf
  ✓ Osi_jll
  ✓ Clp_jll
  ✓ OhMyREPL
  ✓ Cgl_jll
  ✓ Cbc_jll
  ✓ SpecialFunctions
  ✓ DiffRules
  ✓ StatsFuns
  ✓ FastGaussQuadrature
  ✓ ForwardDiff
  ✓ Distributions
  ✓ ADNLPModels
  ✓ NLSolversBase
  ✓ LineSearches
  ✓ NLsolve
  ✓ Zygote
  ✓ Optim
  ✓ GaussianProcesses
  ✓ CUDA
  ✓ MathOptInterface
  ✓ Cbc
  ✓ Ipopt
  ✓ NLopt
  ✓ BayesianOptimization
  ✓ JuMP
  ✓ Hyperopt
  ✓ NLPModelsJuMP
  ✓ Nonconvex
35 dependencies successfully precompiled in 112 seconds (121 already precompiled)


In [2]:
using LinearAlgebra, DataFrames, XLSX, Missings, Test, SpecialFunctions,  SparseArrays , Random, BenchmarkTools, Distributions
using JuMP,Ipopt, NLsolve, Optim, LineSearches
using NLPModels, ADNLPModels, Percival, Nonconvex, Hyperopt
using ForwardDiff
include("IncBetaDer.jl")

Main.IncBetaDer

In [3]:
## load data

xf = XLSX.readxlsx("node_stats_forsimulation_all.xlsx") 
data = vcat( [(XLSX.eachtablerow(xf[s]) |> DataFrames.DataFrame) for s in XLSX.sheetnames(xf)]... ) #for s in XLSX.sheetnames(xf) if (s!="Aggregates Composition" && s!="Dealer Aggregates" && s!="Approx Aggregates")
unique!(data) # delete duplicate rows, use `nonunique(data)` to see if there are any duplicates
data = data[isequal.(data.qt_dt,195), :] # keep quarter == 195 = 2008q4
sort!(data, :assets, rev = true)
units = 1e6;
data[:,[:w, :c, :assets, :p_bar, :b]] .= data[!,[:w, :c, :assets, :p_bar, :b]]./units
# data.b[:] .= missing
# data.c[:] .= missing

col_with_miss = names(data)[[any(ismissing.(col)) for col = eachcol(data)]] # columns with at least one missing
data_nm = coalesce.(data, data.assets/1.5) # replace missing by a value
nm_c = findall(x->x==0,ismissing.(data.c))
nm_b = findall(x->x==0,ismissing.(data.b))
dropmissing(data, [:delta, :delta_alt, :w, :assets, :p_bar]) # remove type missing

names(data) # column names
describe(data)
show(data, allcols = true)

867×12 DataFrame
 Row │ nm_short                      qt_dt  tkr   delta      delta_alt    beta      w          c         assets    nvi_benchmark  p_bar     b        
     │ Any                           Any    Any   Any        Any          Any       Any        Any       Any       Any            Any       Any      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ JPMORGAN CHASE & CO           195    JPM   0.0106652  0.00862097   0.56278   0.168059   1.45164   2.17505   11.0931        2.00699   0.877498
   2 │ CITIGROUP                     195    C     0.0647046  0.0475393    0.481591  0.144022   1.38565   1.93847   11.0931        1.79445   0.930258
   3 │ BANK OF AMER CORP             195    BAC   0.016278   0.0117393    0.50803   0.17871    1.51681   1.82207   11.0931        1.64336   0.808482
   4 │ WELLS FARGO & CO              195    WFC   0.0047069  0.00462975   0.397362  0.

In [4]:
N=5
T = Float64
temp = Array{T}(data[1:N,[:delta, :delta_alt, :w, :assets, :p_bar]])
delta = copy(temp[:,1]); 
delta_alt = copy(temp[:,2]);
w = copy(temp[:,3]); 
assets= copy(temp[:,4]);
p_bar = copy(temp[:,5]);

γ = T(0)
mini_batch = 10;
M = N^2+4*N+mini_batch*N

# z = vcat(A...,b,c,α,β,p...)
# p is N-by-mini_batch

selA  = sparse(1:N^2,1:N^2,T(1),N^2,M)
selb  = sparse(1:N,N^2+1:N^2+N,T(1),N,M)
selc  = sparse(1:N,N^2+N+1:N^2+2N,T(1),N,M)
selα  = sparse(1:N,N^2+2N+1:N^2+3N,T(1),N,M)
selβ  = sparse(1:N,N^2+3N+1:N^2+4N,T(1),N,M)
selp  = sparse(1:mini_batch*N,N^2+4N+1:M,T(1),mini_batch*N,M)

getA(z) = reshape(selA*z,N,N)
getp(z) = reshape(selp*z,N,mini_batch)

getp (generic function with 1 method)

In [34]:
## upper and lower bounds
# low<=z<=upp
uppα = 10
uppβ = 50

low_zero = eps(T) # eps(T) T(0.005)
low = vcat(spzeros(T,N^2),fill(low_zero,N),fill(low_zero,N),ones(T,N),ones(T,N),fill(low_zero,N*mini_batch))
upp = vcat(ones(T,N^2),p_bar,assets,Array{T}(fill(uppα,N)),Array{T}(fill(uppβ,N)),repeat(p_bar,mini_batch))

## linear constraints
# hᵀz=q

h₁₁ᵀ = hcat(sparse(kron(Matrix{T}(I,N,N),p_bar')))
h₁ᵀ = hcat(h₁₁ᵀ,spzeros(T,N,N),sparse(I,N,N),spzeros(T,N,N),spzeros(T,N,N),spzeros(T,N,mini_batch*N))
q₁ = assets

h₂₁ᵀ = repeat(spdiagm(p_bar),1,N)
h₂ᵀ = hcat(h₂₁ᵀ,sparse(I,N,N),spzeros(T,N,N),spzeros(T,N,N),spzeros(T,N,N),spzeros(T,N,mini_batch*N))
q₂ = p_bar

hᵀ = vcat(h₁ᵀ,h₂ᵀ)
q = vcat(q₁,q₂)
function c_lin(z)
    return hᵀ*z .- q
end

## quadratic constraints
# zᵀHz = 0
li = LinearIndices((N,N))
liᵀ= transpose(li)
id = li[tril!(trues(N,N), 0)]
idᵀ = liᵀ[tril!(trues(N,N), 0)]
H₃= sparse(vcat(id,idᵀ),vcat(idᵀ,id),T(0.5),M,M)
h₃ = spzeros(T,M)
q₃ = spzeros(T,1)

H = H₃
function c_quad(z)
    return dot(z,H*z) #transpose(z)*H*z
end

## chance constraints ⟺ c_chance(z)=0
# Prob(c[i]x[i]>=w[i])=delta[i] 
beta_ccdf(a,b,x) = zero(x) < x < one(x) ? one(x) - IncBetaDer.beta_inc_grad(a, b, x)[1] : zero(x)
function c_chance(z)
    zeroz = zero(z[1])+0.0001
    onez = one(z[1])-0.0001
#     display(selc*z)
    return beta_ccdf.(selα*z,selβ*z,max.(min.(w./(selc*z),onez),zeroz) ) .- delta
end

## clearing vector constraint
# quadratic version (ignores min{p_bar,max{0,...}} )
function c_p(z,x)
    c = selc*z
    p = getp(z)
    Aᵀ = transpose(getA(z))
    return reshape((1+γ)*Aᵀ*p + ((1+γ)*(T(1) .- x)).*repeat(c,1,mini_batch)+γ*repeat(p_bar,1,mini_batch)-p,N*mini_batch)
end

# non-linear version
function c_pNL(z,x)
    c = selc*z
    p = getp(z)
    Aᵀ = transpose(getA(z))
    return reshape(min.(max.(0,(1+γ)*Aᵀ*p + ((1+γ)*(T(1) .- x)).*repeat(c,1,mini_batch)+γ*repeat(p_bar,1,mini_batch)),p_bar)-p,N*mini_batch)
end

c_pNL (generic function with 1 method)

In [35]:
# write constraints as cL<=c(z)<=cU, setting cL=cU for equality constraints
cL = spzeros(3N+1+mini_batch*N)
cU = cL
function c(z)
    #x = rand(N,mini_batch)
    x = ones(N,mini_batch)/10
    vcat(c_lin(z),
        c_quad(z),
        c_chance(z),
        c_p(z,x)
        )
end

# objective to max or min
function obj(z)
    c = selc*z
    α = selα*z
    β = selβ*z
    p = getp(z)
    return sum(c.*α./(α + β))-sum(p)
end

# write constraints as cL<=c(z)<=cU, setting cL=cU for equality constraints
# cL = spzeros(3N+1)
# cU = cL
# function c(z)
#     vcat(hᵀ*z .- q,
#         dot(z,H*z),
#         beta_ccdf.(selα*z,selβ*z,w./(selc*z)) .- delta
#         )
# end
# objective to max or min
# function obj(z)
#     Aᵀ = transpose(getA(z))
#     c = selc*z
#     α = selα*z
#     β = selβ*z
#     p = getp(z)
#     d = Beta.(ones(N),2*ones(N))
#     x = transpose(reduce(hcat,rand.(d, mini_batch)))
#     return sum(c.*α./(α + β))-sum(min.(max.(0,(1+γ)*Aᵀ*p + ((1+γ)*(T(1) .- x)).*repeat(c,1,mini_batch)+γ*repeat(p_bar,1,mini_batch))))
# end

function obj_full(z)
    return obj(z)+sum(p_bar)
end

obj_full (generic function with 1 method)

In [36]:
@testset "Problem specified correctly" begin
    num_tests = 10
    for nn=1:num_tests
        z_test = Array((low+min.(upp,100))/2)
        x_test = rand(N,mini_batch)

        A_test = getA(z_test)
        b_test = selb*z_test
        c_test = selc*z_test
        α_test = selα*z_test
        β_test = selβ*z_test
        p_test = getp(z_test)

        @testset "Upper and lower bounds" begin
            @test all(getA(low).==0)
            @test all(getA(upp).==1)
            @test all(selb*low.==low_zero)
            @test all(selb*upp.==p_bar)
            @test all(selc*low.==low_zero)
            @test all(selc*upp.==assets)
            @test all(selα*low.==1)
            @test all(selα*upp.==uppα)
            @test all(selβ*low.==1)
            @test all(selβ*upp.==uppβ)
            @test all(getp(low).==low_zero)
            @test all(getp(upp).==p_bar)      
        end

        @testset "z_test respects upper and lower bounds" begin
            @test all(0 .<= A_test .<= 1)
            @test all(0 .<= b_test .<= p_bar)
            @test all(0 .<= c_test .<= assets)
            @test all(1 .<= α_test .<= uppα)
            @test all(1 .<= β_test .<= uppβ)
            @test all(0 .<= p_test .<= p_bar)
        end

        @testset "Linear constraints" begin
            l1 = [sum(p_bar[j]*A_test[j,i] for j=1:N)-assets[i]+c_test[i] for i=1:N] # payments to $i$ from other nodes $j$ add up to inside assets $d= assets - c$
            l2 = [p_bar[i]*sum(A_test[i,j] for j=1:N)-p_bar[i]+b_test[i] for i=1:N] # payments from $i$ to other nodes add up to inside liabilities $f = p_bar - b$  
            @test vcat(l1,l2) ≈ c_lin(z_test)
        end
        @testset "a[i,j]*a[j,i]=0" begin
                @test sum(A_test[i,j]*A_test[j,i] for i=1:N for j=1:i) ≈ c_quad(z_test) 
        end
        @testset "Prob(c[i]x[i]>=w[i])=delta[i]" begin
            @test [ccdf(Beta(α_test[i],β_test[i]),w[i]/c_test[i]) - delta[i] for i=1:N] ≈ c_chance(z_test)
        end
        @testset "Clearing vector p(x)" begin
            @test [(1+γ)*sum(A_test[j,i]*p_test[j,q] for j=1:N)+(1+γ)*(1-x_test[i,q])*c_test[i]-γ*p_bar[i]-p_test[i,q] for i=1:N, q=1:mini_batch] ≈ reshape(c_p(z_test,x_test),N,mini_batch)
            @test [min(p_bar[i],max((1+γ)*sum(A_test[j,i]*p_test[j,q] for j=1:N)+(1+γ)*(1-x_test[i,q])*c_test[i]-γ*p_bar[i],0))-p_test[i,q] for i=1:N, q=1:mini_batch] ≈ reshape(c_pNL(z_test,x_test),N,mini_batch)
        end
    end
end


Test Summary:               | Pass  Total
Problem specified correctly |  230    230


Test.DefaultTestSet("Problem specified correctly", Any[Test.DefaultTestSet("Upper and lower bounds", Any[], 12, false, false), Test.DefaultTestSet("z_test respects upper and lower bounds", Any[], 6, false, false), Test.DefaultTestSet("Linear constraints", Any[], 1, false, false), Test.DefaultTestSet("a[i,j]*a[j,i]=0", Any[], 1, false, false), Test.DefaultTestSet("Prob(c[i]x[i]>=w[i])=delta[i]", Any[], 1, false, false), Test.DefaultTestSet("Clearing vector p(x)", Any[], 2, false, false), Test.DefaultTestSet("Upper and lower bounds", Any[], 12, false, false), Test.DefaultTestSet("z_test respects upper and lower bounds", Any[], 6, false, false), Test.DefaultTestSet("Linear constraints", Any[], 1, false, false), Test.DefaultTestSet("a[i,j]*a[j,i]=0", Any[], 1, false, false)  …  Test.DefaultTestSet("Linear constraints", Any[], 1, false, false), Test.DefaultTestSet("a[i,j]*a[j,i]=0", Any[], 1, false, false), Test.DefaultTestSet("Prob(c[i]x[i]>=w[i])=delta[i]", Any[], 1, false, false), Test.D

In [37]:
z_test = Array((low+min.(upp,100))/2)
c(z_test)

66-element Vector{Float64}:
 2.7548112500000004
 2.8731022500000005
 2.93130325
 3.1875177500000005
 3.30145075
 4.013986000000001
 3.588896
 3.2867150000000005
 2.4146460000000003
 2.065106
 3.75
 0.5843624880599494
 0.5660320288942458
 ⋮
 1.9068446750000005
 1.8916899750000007
 1.8964455250000003
 1.8962561250000003
 1.9194204750000003
 1.9068446750000005
 1.8916899750000007
 1.8964455250000003
 1.8962561250000003
 1.9194204750000003
 1.9068446750000005
 1.8916899750000007

In [38]:
lowA = Array(low)
uppA = Array(upp)
z_testA = Array(z_test)
cA(z) = sum(abs2,Array(c(z)))

cA (generic function with 1 method)

In [39]:
z0= Array((low+min.(upp,100))/10)
m_lin = Nonconvex.Model(z->0.0)
addvar!(m_lin,lowA,uppA)
add_eq_constraint!(m_lin,z->c_lin(z))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions()
r_lin = Nonconvex.optimize(m_lin, alg,z0, options = options)
tol = 1e-16
@test r_lin.minimum ≈ 0 
@test sum(abs2,c_lin(r_lin.minimizer)) < tol


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      950
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:       10
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Test Passed

In [40]:
z0=r_lin.minimizer
m_quad = Nonconvex.Model(z->0.0)
addvar!(m_quad,lowA,uppA)
add_eq_constraint!(m_quad,z->c_quad(z))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions()
r_quad = Nonconvex.optimize(m_quad, alg,z0, options = options)

tol = 1e-16
@test r_quad.minimum ≈ 0 
@test sum(abs2,c_quad(r_quad.minimizer)) < tol


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       95
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Test Passed

In [41]:
z0 = (r_lin.minimizer+r_quad.minimizer)/2

m_chance = Nonconvex.Model(z->0.0)
addvar!(m_chance,lowA,uppA)
add_eq_constraint!(m_chance,z->c_chance(z))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions()
r_chance = Nonconvex.optimize(m_chance, alg, z0, options = options)

tol = 1e-16
@test r_chance.minimum ≈ 0 
@test sum(abs2,c_chance(r_chance.minimizer)) < tol

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      475
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Test Passed

In [42]:
z0 = (r_lin.minimizer+r_quad.minimizer+r_chance.minimizer)/3
x0 = rand(N,mini_batch)

m_p = Nonconvex.Model(z->0.0)
addvar!(m_p,lowA,uppA)
add_eq_constraint!(m_p,z->c_p(z,x0))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions(constr_viol_tol = 1e-16 )
r_p = Nonconvex.optimize(m_p, alg, z0, options = options)

tol = 1e-16
@test r_p.minimum ≈ 0 
@test sum(abs2,c_p(r_p.minimizer,x0)) < tol

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4750
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:       50
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

Test Passed

In [44]:
z0 = r_p.minimizer #(r_lin.minimizer+r_quad.minimizer+r_chance.minimizer+r_p.minimizer)/4

m = Nonconvex.Model(z->0.0)
addvar!(m,lowA,uppA)
add_eq_constraint!(m,z->c(z))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions()
r = Nonconvex.optimize(m, alg, z0, options = options)

tol = 1e-16
@test r.minimum ≈ 0 
@test sum(abs2,c(r.minimizer)) < tol

@test sum(abs2,c_lin(r.minimizer)) < tol
@test sum(abs2,c_quad(r.minimizer)) < tol
@test sum(abs2,c_chance(r.minimizer)) < tol
@test sum(abs2,c_p(r.minimizer,ones(N,mini_batch)/10)) < tol

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     6270
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:       66
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

LoadError: [91mThere was an error during testing[39m

In [ ]:
z0 = r.minimizer

m = Nonconvex.Model(z->0.0)
addvar!(m,lowA,uppA)
add_eq_constraint!(m,z->c(z))

alg = IpoptAlg()
options = Nonconvex.IpoptOptions(constr_viol_tol = 1e-16 , max_iter=4000)
r = Nonconvex.optimize(m, alg, z0, options = options)

tol = 1e-16
@test r.minimum ≈ 0 
@test sum(abs2,c(r.minimizer)) < tol

@test sum(abs2,c_lin(r.minimizer)) < tol
@test sum(abs2,c_quad(r.minimizer)) < tol
@test sum(abs2,c_chance(r.minimizer)) < tol
@test sum(abs2,c_p(r.minimizer,ones(N,mini_batch)/10)) < tol

In [43]:
Pkg.add("JLD2")
using JLD2
A_c = getA(r.minimizer)
b_c = selb*r.minimizer
c_c = selc*r.minimizer
α_c = selα*r.minimizer
β_c = selβ*r.minimizer
p_c = getp(r.minimizer)

jldsave("feasible_sol.jld2"; A_c, b_c, c_c, α_c, β_c, p_c, r)

display(A_c)
display(b_c)
display(c_c)
display(α_c)
display(β_c)
display(p_c)

In [43]:
z0 = r_p.minimizer #(r_lin.minimizer+r_quad.minimizer+r_chance.minimizer+r_p.minimizer)/4

m = Nonconvex.Model(z->0.0)
addvar!(m,lowA,uppA)
add_eq_constraint!(m,z->c(z))

alg = HyperoptAlg(IpoptAlg())
sampler = Nonconvex.RandomSampler()
options = HyperoptOptions(
    sub_options = IpoptOptions(first_order = true),
    sampler = sampler,
    keep_all = true,
)
r = Nonconvex.optimize(m, alg, z0, options = options)

tol = 1e-16
@test r.minimum ≈ 0 
@test sum(abs2,c(r.minimizer)) < tol

@test sum(abs2,c_lin(r.minimizer)) < tol
@test sum(abs2,c_quad(r.minimizer)) < tol
@test sum(abs2,c_chance(r.minimizer)) < tol
@test sum(abs2,c_p(r.minimizer,ones(N,mini_batch)/10)) < tol

┌ Info: Searching starting point... 
└ @ Nonconvex /home/ec2-user/.julia/packages/Nonconvex/JzrTb/src/wrappers/hyperopt.jl:86


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     6270
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       95
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       95
                     variables with only upper bounds:        0
Total number of equality constraints.................:       66
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

LoadError: InterruptException:

In [ ]:
Optim.optimize(cA,lowA,uppA ,z_testA, SAMIN(verbosity=1), Optim.Options(iterations=10^6))

In [ ]:
#inner_optimizer = GradientDescent()
inner_optimizer = BFGS() #GradientDescent(linesearch=LineSearches.BackTracking(order=3))
results = optimize(cA,lowA,uppA ,z_testA, Fminbox(inner_optimizer),  Optim.Options(g_tol = 1e-12,iterations = 10); autodiff = :forward)

In [ ]:
getA(Optim.minimizer(results))

In [ ]:
nlp = ADNLPModel(T(0), z_testA, lowA, uppA, c, Array(cL), Array(cU))

In [ ]:
import NLopt

alg = NLoptAlg(:LD_MMA)
options = Nonconvex.NLoptOptions()
r = optimize(m, alg, [1.234, 2.345], options = options)
r.minimum
r.minimizer

In [ ]:
rng =  Random.seed!(123)
z0 = (low+upp)/2
x1 = ones(T,N)/10 #rand(T, N)
# Ex = ones(T,N)/10 #ones(T,N)/2 # expectation of x# convert from sparse into full arrays
obj_full(z0)

In [ ]:
z0 = (low+min.(upp,100))/2
@show obj(z0)
@show c(z0)-cL
@show (low,upp)

In [ ]:
(obj(z0),obj(z0),obj(z0),obj(z0))


In [ ]:
linear = 1:2N
quadratic = 2N+1
nlp=ADNLPModel(obj,z0,Array(low),Array(upp),c,Array(cL),Array(cU); lin=linear)

z0=Array(z0);cz0=Array(c(z0));
nlp_Lag=AugLagModel(nlp, ones(T,length(cz0)), 10.0, z0,cz0)

In [ ]:
using JSOSolvers
tron(nlp_Lag,max_time=60.0*5)

In [ ]:
# ccdf of a beta distribution 
function dist_cdf(p...)
    #α,β,wc = p[1], p[2], p[3]
    obj(x,p) = (x>=p[3])*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x)
    return Quad1d(obj,p)
end
function Quad1d(obj,p)
    prob = QuadratureProblem(obj,0,1,p,order=5)
    return Quadrature.solve(prob,QuadGKJL(),reltol = 1e-5)[1]
end
function find_parametric_optimum(g, a)
    optimize(x -> g(x, a), ...)
end

In [ ]:
@variable(m, lowA[i]<=z[i=1:M]<=uppA[i]) 
@variable(m, 1.0<=α[i=1:N]<=2.0, start = 4.1) 
@variable(m, 1.0<=β[i=1:N]<=150.0, start = 30.0) 
@constraint(m, hᵀA*z.==qA) 
@constraint(m,transpose(z)*HA*z==0.0) 
@constraint(m,px[i=1:N],transpose(z)*H₄A[i]*z.==h₄₄₄ᵀA[i]*z.- q₄A[i] )
@constraint(m,z[diag(LinearIndices(ones(N,N)))].==0)

# match probabilities of default

JuMP.register(m, :dist_cdf, 3, dist_cdf; autodiff=true)
@NLexpression(m,wc[i=1:N],w[i]/z[N^2+N+i]) #c[i] = z[N^2+N+i]
for i in 1:N
    @eval ($(Symbol("vv$i"))) = [α[$i],β[$i],wc[$i]]
    @eval @NLconstraint(m,dist_cdf(($(Symbol("vv$i")))...)== delta[$i]) 
end

ex = @expression(m,(h₀ᵀA*z)[1])
@objective(m, Max,  ex )

In [ ]:
push!(LOAD_PATH,"/home/ec2-user/SageMaker/IncBetaDer/");
using IncBetaDer

In [ ]:
display(LOAD_PATH)

In [ ]:
using Distributions, DistributionsAD, ForwardDiff, Quadrature
push!(LOAD_PATH,"/home/ec2-user/SageMaker/IncBetaDer/");
using IncBetaDer

In [ ]:
function dist_cdf(a, b, y)
    #α,β,wc = p[1], p[2], p[3]
    obj(x,p) = (x>=y)*DistributionsAD.pdf(DistributionsAD.Beta(p[1],p[2]),x)
    return Quad1d(obj,hcat(a, b))
end
function Quad1d(obj,p)
    prob = QuadratureProblem(obj,0,1,p,order=100)
    return Quadrature.solve(prob,QuadGKJL(),reltol = 1e-16,abstol = 1e-16)[1]
end

a=1.1;b=2.1;x=0.1;
z = [a, b, x]

In [ ]:
@show dist_cdf(z...)
@show 1 .- IncBetaDer.beta_inc_grad(a, b, x,1e10, 1,1e-20)[1]

In [ ]:
@show ForwardDiff.derivative(α->dist_cdf(α, b, x),a)[1]
@show -IncBetaDer.beta_inc_grad(a, b, x,1e10, 1,1e-20)[2];

In [ ]:
@show ForwardDiff.derivative(β->dist_cdf(a,β, x),b)[1]
@show -IncBetaDer.beta_inc_grad(a, b, x)[3];

In [ ]:
@show ForwardDiff.derivative(y->dist_cdf(a,b,y),x)[1]
@show -IncBetaDer.beta_inc_grad(a, b, x,1e9, 1,1e-16)[4];

In [ ]:
ee=1e-16
(dist_cdf(a,b,x+ee)-dist_cdf(a,b,x-ee))/(2ee)

In [ ]:
N=3;Q=2
rowInd = repeat(1:N^2,Q)
colInd = [hcat([repeat((1:N).+q*N,N) for q=0:Q-1]...)...]

matInd=hcat(vcat(rowInd .+ N^2 .+ 4N, rowInd),vcat(colInd,colInd .+ N^2 .+ 1))
[matInd[(1:N) .+ q*N,1] for q=1:Q]
[matInd[(1:N) .+ q*N,2] for q=1:Q]


In [ ]:
rowIndH=collect(Iterators.partition(vcat(rowInd .+ N^2 .+ 4N, rowInd),N))
colIndH=collect(Iterators.partition(vcat(colInd,colInd .+ N^2 .+ 1),N))

In [ ]:
(rowInd,colInd .+ N^2 .+ 1)

In [ ]:
mat =[sparse(rowIndH[i+N*Q],colIndH[i+N*Q],ones(N)/2,N^2+4N+Q*N,N^2+4N+Q*N) for i=1:N*Q];
matsp=sparse(Symmetric(mat[6],:U))

In [ ]:
ind1(q)= q*N .+ (1:N)
ind2(q)= N^2+4N +q*N .+ (1:N)

([vcat(ind1(q),ind2(q+i)) for q=0,i=1],[vcat(ind2(q+i),ind1(q)) for q=0,i=1])

In [ ]:
N^2+4*N

In [ ]:
ind1(0),ind2(0)
ind1(1),ind2(0)
ind1(2),ind2(0)
ind1(0),ind2(1)
ind1(1),ind2(1)
ind1(2),ind2(1)

ind2(0),ind1(0)
ind2(0),ind1(1)
ind2(0),ind1(2)
ind2(1),ind1(0)
ind2(1),ind1(1)
ind2(1),ind1(2)

In [ ]:
ind1(q)= q*N .+ (1:N)
ind2(q)= N^2+4N +q*N .+ (1:N)
H = [sparse(vcat(ind1(i),ind2(q)),vcat(ind2(q),ind1(i)),ones(2*N)/2,N^2+4N+Q*N,N^2+4N+Q*N) for i=0:N-1,q=0:Q-1];
using Test
@test [transpose(z)*H[i,q]*z for i=1:N,q=1:Q]≈transpose(A)*p

In [ ]:
A = rand(3,3);
x = rand(4N);
p = rand(N,Q);

z = vcat(A...,x...,p...)
(transpose(z)*H[1,1]*z,dot(A[:,1],p[:,1]))


[(transpose(z)*H[i,q]*z,dot(A[:,i],p[:,q])) for i=1:N,q=1:Q]